In [15]:
# Importing necessary packages for web scrapping 
from bs4 import BeautifulSoup
import requests
import pandas as pd

#### Extracting One Piece Character Names

In [66]:
# Define the URL of the webpage to scrape  
URL = "https://onepiece.fandom.com/wiki/List_of_Canon_Characters"
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html.parser')

# Locate the table 
table = soup.find('table', attrs={'class': 'fandom-table sortable'})
characters_list = []

# Extract names from the table
rows = table.find_all('tr')
for row in rows[1:]:
    cells = row.find_all(['td'])
    row_data = [cell.get_text(strip=True) for cell in cells]
    name = row_data[1]  #Only get data 'name'
    characters_list.append(name)

In [68]:
# Import names into dataframe
df = pd.DataFrame(characters_list, columns=["Name"])

In [69]:
df.head()

,Name
0,A O
1,Abdullah
2,Absalom
3,Acilia
4,Adele


#### Extracting One Piece Character Details

In [70]:
name_list = []

for name in df['Name'].tolist():
    formatted_name = name.replace(' ', '_')
    
    URL = "https://onepiece.fandom.com/wiki/" + formatted_name
    try:
        r = requests.get(URL)
        r.raise_for_status()
        
        soup = BeautifulSoup(r.content, 'html.parser')
        table_statistics = soup.find('aside', attrs={'class': 'portable-infobox'}).findAll('section')
        
        # Character Data
        character_data = {}
        for section in table_statistics[:1]:
            for item in section.findAll('div', {'class': 'pi-item'}):
                label = item.find('h3', class_='pi-data-label').text.strip()
                value = item.find('div', class_='pi-data-value').text.strip()
                character_data[label] = value
        
        # Devil Fruit Data
        devil_fruit_data = {}
        for section in table_statistics[1:]:
            for item in section.findAll('div', {'class': 'pi-item'}):
                label = 'Devil Fruit ' + item.find('h3', class_='pi-data-label').text.strip()
                value = item.find('div', class_='pi-data-value').text.strip()
                devil_fruit_data[label] = value
        
        # Merge character_data and devil_fruit_data
        all_data = {**character_data, **devil_fruit_data}
        
        data_details = {name: all_data}
        name_list.append(data_details)
        
    except requests.exceptions.HTTPError as e:
        print("Error " + name)

    except Exception as e:
        print("An unexpected error occurred for {}: {}".format(name, str(e)))
        
print("Finished scrapping") 

Error Jack-in-the-Box
An unexpected error occurred for Minatomo: 'NoneType' object has no attribute 'findAll'
An unexpected error occurred for Minatomo: 'NoneType' object has no attribute 'findAll'
Error Mr. Sacrifice
An unexpected error occurred for : 'NoneType' object has no attribute 'findAll'
An unexpected error occurred for Ryuma: 'NoneType' object has no attribute 'findAll'
Error Wall Zombie
Finished scrapping


In [72]:
# Print the first item
print(name_list[0])

{'A O': {'Japanese Name:': 'A・O', 'Romanized Name:': 'Ā Ō', 'Official English Name:': 'A.O. (VIZ Media and FUNimation subs);A.O (FUNimation dub)', 'Debut:': 'Chapter 551; Episode 460[1]', 'Affiliations:': 'A O Pirates;[2] Subordinate of the Whitebeard Pirates[1][2]', 'Occupations:': 'Pirate; Captain[1][2]', 'Status:': 'Unknown', 'Birthday:': 'January 15th[3]', 'Japanese VA:': 'Kōhei Fukuhara', 'Funi English VA:': 'Mark Fickert'}}


In [75]:
# Create a dataframe with the combined data.
OnePiece = pd.concat([pd.DataFrame(result) for result in name_list], axis=1)
OnePiece = OnePiece.transpose()
OnePiece = OnePiece.rename_axis('Name').reset_index()

OnePiece.head(3)

,Name,Japanese Name:,Romanized Name:,Official English Name:,Debut:,Affiliations:,Occupations:,Status:,Birthday:,Japanese VA:,...,Meaning:,Fruit Debut:,Usage Debut:,Type:,Previous User:,Features:,Homeland:,Literal Meaning:,User(s):,Fighting Style Focus:
0,A O,A・O,Ā Ō,A.O. (VIZ Media and FUNimation subs);A.O (FUNi...,Chapter 551; Episode 460[1],A O Pirates;[2] Subordinate of the Whitebeard ...,Pirate; Captain[1][2],Unknown,January 15th[3],Kōhei Fukuhara,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Abdullah,アブドーラ,Abudōra,Abdullah,Chapter 704; Episode 633[1],Ideo Pirates[2][3]; Straw Hat Grand Fleet[4],"Pirate[3], Criminal; Bounty Hunter (former)[1]",Alive,NaN,Keiji Hirai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Absalom,アブサロム,Abusaromu,Absalom,Chapter 444; Episode 339[1],Thriller Bark Pirates (Mysterious Four)[2] (fo...,Reporter[3]Soldier Zombie and General Zombie L...,Deceased,December 30th[6],Hiroaki Miura,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
# Save dataframe to csv file
csv_file = "onepiece_character_detail.csv"
OnePiece.to_csv(csv_file, index=False)